In [19]:
import pandas as pd
import re
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

### Create a Pandas dataFrame

In [20]:
data = {
    'organization': ['Cognizant Technology Solutions', 'Aakarist Pvt Ltd', 'WNS Global Services'],

    'designation': ['Reporting and Data Analyst', 'Senior Analyst', 'Analyst'],

    'start_date': ['2022-04-01', '2020-09-01', '2019-02-01'],

    'end_date': ['2024-10-01', '2022-03-01', '2020-09-01']
    }

# Convert to DataFrame
df = pd.DataFrame(data)

### Tenure calculation and mapping

In [21]:
def calculate_and_map_tenure(df, start_date_col, end_date_col, organization_col, tenure_mapping):
    """
    Calculate the tenure in months, convert to integer, and map it based on the organization name.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    start_date_col (str): The column name for the start date.
    end_date_col (str): The column name for the end date.
    organization_col (str): The column name for the organization name.
    tenure_mapping (dict): A dictionary mapping organization names to mapped tenure values.

    Returns:
    pd.DataFrame: The DataFrame with the calculated tenure in months and mapped tenure values.
    """
    # Convert start_date and end_date columns to datetime format
    df[start_date_col] = pd.to_datetime(df[start_date_col])
    df[end_date_col] = pd.to_datetime(df[end_date_col])

    # Calculate the tenure in months and convert to integer
    df['tenure_months'] = ((df[end_date_col] - df[start_date_col]).dt.days / 30.44).astype(int)

    # Apply the tenure mapping based on organization name
    df['mapped_tenure'] = df[organization_col].map(tenure_mapping)

    return df

tenure_mapping = {
    'Cognizant Technology Solutions': 'Senior Role',
    'Aakarist Pvt Ltd': 'Mid-Level Role',
    'WNS Global Services': 'Junior Role',
}

# Usage
df = calculate_and_map_tenure(df, 'start_date', 'end_date', 'organization', tenure_mapping)

df.head(5)

,organization,designation,start_date,end_date,tenure_months,mapped_tenure
0,Cognizant Technology Solutions,Reporting and Data Analyst,2022-04-01,2024-10-01,30,Senior Role
1,Aakarist Pvt Ltd,Senior Analyst,2020-09-01,2022-03-01,17,Mid-Level Role
2,WNS Global Services,Analyst,2019-02-01,2020-09-01,18,Junior Role


In [22]:
responsibilities_mapping = {
    'Cognizant Technology Solutions': [
        "Led the analysis and reporting of key business metrics, ensuring data accuracy.",
        "Collaborated with cross-functional teams to identify data requirements.",
        "Developed automated reporting solutions, reducing manual workload by 50%."
    ],
    'Aakarist Pvt Ltd': [
        "Conducted in-depth data analysis to drive business insights.",
        "Optimized data collection processes, improving efficiency by 30%.",
        "Presented findings to senior management, influencing strategic decisions."
    ],
    'WNS Global Services': [
        "Performed data analysis to support decision-making.",
        "Maintained data integrity and accuracy across multiple datasets.",
        "Provided actionable insights to stakeholders based on data trends."
    ]
}

# Map the responsibilities to the DataFrame
df['responsibilities'] = df['organization'].map(responsibilities_mapping)


In [23]:
df.head(10)

,organization,designation,start_date,end_date,tenure_months,mapped_tenure,responsibilities
0,Cognizant Technology Solutions,Reporting and Data Analyst,2022-04-01,2024-10-01,30,Senior Role,[Led the analysis and reporting of key busines...
1,Aakarist Pvt Ltd,Senior Analyst,2020-09-01,2022-03-01,17,Mid-Level Role,[Conducted in-depth data analysis to drive bus...
2,WNS Global Services,Analyst,2019-02-01,2020-09-01,18,Junior Role,[Performed data analysis to support decision-m...


### Export the dataFrame to csv and Pickle

In [24]:
def export_dataframe(df, pickle_file_path, csv_file_path):
    """
    Export a DataFrame to both a pickle file and a CSV file.

    Parameters:
    df (pd.DataFrame): The DataFrame to export.
    pickle_file_path (str): The file path where the pickle file will be saved.
    csv_file_path (str): The file path where the CSV file will be saved.
    """
    # Export to pickle file
    df.to_pickle(pickle_file_path)
    print(f"DataFrame exported to pickle file: {pickle_file_path}")

    # Export to CSV file
    df.to_csv(csv_file_path, index=False)
    print(f"DataFrame exported to CSV file: {csv_file_path}")


# Assume df is your DataFrame
pickle_path = 'data_work_experience.pkl'
csv_path = 'data_work_experience.csv'

export_dataframe(df, pickle_path, csv_path)


DataFrame exported to pickle file: data_work_experience.pkl
DataFrame exported to CSV file: data_work_experience.csv


### Load data from a Pickle file

In [39]:

def load_data(file_path):
    """
    Load data from a CSV file.

    Parameters:
    file_path (str): The path to the CSV file.

    Returns:
    df (pd.DataFrame): The loaded DataFrame.
    """
    df = pd.read_pickle(file_path)
    return df

file_path = 'data_work_experience.pkl'
df = load_data(file_path)

df.head(3)

,organization,designation,start_date,end_date,tenure_months,mapped_tenure,responsibilities
0,Cognizant Technology Solutions,Reporting and Data Analyst,2022-04-01,2024-10-01,30,Senior Role,[Led the analysis and reporting of key busines...
1,Aakarist Pvt Ltd,Senior Analyst,2020-09-01,2022-03-01,17,Mid-Level Role,[Conducted in-depth data analysis to drive bus...
2,WNS Global Services,Analyst,2019-02-01,2020-09-01,18,Junior Role,[Performed data analysis to support decision-m...


### Custom hover text function

In [40]:
def generate_custom_hover(row):
    responsibilities_text = '<br>'.join(row.get('responsibilities', []))  # Handle missing 'responsibilities' key
    return (
        f"Organization: {row.get('organization', 'N/A')}<br>"
        f"Designation: {row.get('designation', 'N/A')}<br>"
        f"Start Date: {row.get('start_date', 'N/A')}<br>"
        f"End Date: {row.get('end_date', 'N/A')}<br>"
        f"Tenure (Months): {row.get('tenure_months', 'N/A')}<br>"
        f"Job Level: {row.get('mapped_tenure', 'N/A')}<br>"
        f"Responsibilities:<br>{responsibilities_text}"
    )

# Apply the function to generate hover text
df['hover_template'] = df.apply(generate_custom_hover, axis=1)


### Color discrete map

In [41]:
def create_color_map(df, category_column):
    """
    Create a color map for unique values in a specified column.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    category_column (str): The column name with categorical data.

    Returns:
    color_discrete_map (dict): A dictionary mapping unique categories to colors.
    """
    # Extract unique categories
    unique_values_list = df[category_column].unique().tolist()
    
    # Define a color map
    color_map = px.colors.sequential.Jet[:len(unique_values_list)]
    color_discrete_map = dict(zip(unique_values_list, color_map))
    
    return color_discrete_map

# Assuming df is your DataFrame and 'category' = 'mapped_tenure' is the column with unique values
color_discrete_map = create_color_map(df, 'tenure_months')

### Function to create, update and save plots

In [42]:
def prepare_and_save_plot(fig, width=600, height=600, template='plotly_dark', 
                          plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', 
                          font_family="Open Sans, sans-serif", font_size=10, font_color="white", 
                          hover_template=None,  # Changed to None by default
                          base_filename='plot', auto_open=False, counter=1, 
                          title=None, xaxis_title=None, yaxis_title=None,
                          legend_title=None, coloraxis_colorbar=None):  # Added parameter for legend title
    """
    Update the layout of the Plotly figure, set the hover template, and save the plot to an HTML file
    with an automatically incremented filename.

    Parameters:
    fig (go.Figure): The Plotly figure to be updated.
    width (int): Width of the figure. Default is 600.
    height (int): Height of the figure. Default is 600.
    template (str): Plotly template for the figure. Default is 'plotly_dark'.
    plot_bgcolor (str): Background color of the plot area. Default is transparent.
    paper_bgcolor (str): Background color of the paper area. Default is transparent.
    font_family (str): Font family for the text. Default is 'Open Sans, sans-serif'.
    font_size (int): Font size for the text. Default is 10.
    font_color (str): Font color for the text. Default is 'white'.
    hover_template (str): Custom hover template for traces. Default is None.
    base_filename (str): Base name for the file. Default is 'plot'.
    auto_open (bool): Whether to open the file in the browser after saving. Default is False.
    counter (int): Counter for incrementing filenames. Default is 1.
    title (str): Title of the plot. Default is None.
    xaxis_title (str): Title for the x-axis. Default is None.
    yaxis_title (str): Title for the y-axis. Default is None.
    legend_title (str): Title for the legend. Default is None.

    Returns:
    fig (go.Figure): The updated Plotly figure.
    """
    # Update layout
    fig.update_layout(
        width=width,
        height=height,
        template=template,
        plot_bgcolor=plot_bgcolor,
        paper_bgcolor=paper_bgcolor,
        font=dict(
            family=font_family,
            size=font_size,
            color=font_color
        ),
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        legend_title=legend_title,
        coloraxis_colorbar=dict(
            title=coloraxis_colorbar
        ),
    )
    
    # Set custom hover template if provided
    if hover_template:
        fig.update_traces(hovertemplate=hover_template)
    
    # Generate filename with counter
    filename = f'{base_filename}_{counter}.html'
    
    # Save the figure to an HTML file
    pio.write_html(fig, file=filename, auto_open=auto_open)
    
    return fig


In [43]:
def create_timeline_plot(df, start_date, end_date, y_axis, color, text, hover_text=None):
    """
    Create a timeline plot using Plotly Express with an optional y-axis.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    start_date (str): The column name for the start date of the timeline.
    end_date (str): The column name for the end date of the timeline.
    color (str): The column name for the color coding of the timeline entries.
    text (str): The column name for text to display on the timeline (e.g., job titles).
    y_axis (str): The column name for the y-axis to categorize the entries.
    hover_text (str): Optional column name for custom hover text. Default is None.

    Returns:
    fig (go.Figure): The Plotly timeline figure.
    """
    # Create the timeline plot
    fig = px.timeline(
        df,
        x_start=start_date,
        x_end=end_date,
        y=y_axis,
        color=color,
        text=text,
        title="Timeline Plot"
    )
    
    # Update hover template if hover_text is provided
    if hover_text:
        fig.update_traces(
            hovertemplate=df[hover_text].apply(lambda x: f"{x}<extra></extra>")
        )
    
    # Return the figure without additional layout customizations
    return fig

### Creating and updating the Sunburst plot for the entire skills section

In [51]:
df_sorted = df.sort_values(by='start_date')

fig = create_timeline_plot(
    df_sorted,
    start_date='start_date',
    end_date='end_date',
    y_axis='organization',
    color='tenure_months',
    text='designation',
    hover_text='hover_template'
)

# Prepare and save the plot
prepare_and_save_plot(
    fig,
    width=1200,
    height=282.5,
    #title="My Work Timeline",
    xaxis_title="Start Date",
    #yaxis_title="Organizations",
    coloraxis_colorbar='Tenure',
    legend_title='Job Level',
    base_filename='timeline',
    counter='work_exp'
)